# 1. Renderizar OpenAI Gym en Colab


In [1]:
install=True
if install:
  !pip install gym pyvirtualdisplay
  !apt-get install -y xvfb python-opengl ffmpeg
  !apt-get update
  !apt-get install cmake
  !pip install --upgrade setuptools
  !pip install ez_setup
  !pip install -U gym>=0.21.0
  !pip install -U gym[atari,accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 56 not upgraded.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_

# **¡¡¡¡¡REINICIAR ENTORNO!!!!**

# 2. Primera partida

In [18]:
# Funciones para mostrar
import gym
# from gym.wrappers import Monitor
from gym.wrappers.monitoring.video_recorder import VideoRecorder
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
import os
import uuid

def show_video():
  mp4list = sorted(glob.glob('video/*.mp4'), key=os.path.getmtime)
  for mp4 in mp4list:
    # mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<h1>{1}</h1><video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'),mp4)))
  else: 
    pass
    

def wrap_env(env):
  import os
  id=f'./video/{uuid.uuid1()}.mp4'
  print(os.getcwd(),id)
  os.makedirs("./video",exist_ok=True)
  # env = Monitor(env,id , force=False)
  video_recorder = VideoRecorder(env, id, enabled=True)
  return env, video_recorder

### Entorno

In [2]:
import gym

def query_environment(name):
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space.shape}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

In [3]:
query_environment("BreakoutDeterministic-v4")

Action Space: Discrete(4)
Observation Space: (210, 160, 3)
Max Episode Steps: 100000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


In [4]:
!rm -rf video
env, video_recorder= wrap_env(gym.make("BreakoutDeterministic-v4",render_mode="human"))
 

observation = env.reset()

while True:
  
    # env.render()
    video_recorder.capture_frame()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break;

video_recorder.close()            
env.close()



/content ./video/ba087dce-dc3a-11ec-85b2-0242ac1c0002.mp4


/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitoring/video_recorder.py:422: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np.__version__


In [19]:
show_video()


# 3. Preparando NN Atari Breakout

In [6]:
from abc import ABCMeta, abstractmethod


class GameInterface(metaclass=ABCMeta):
    @property
    def action_space(self):
        return NotImplementedError

    @property
    def state(self):
        return NotImplementedError

    @property
    def total_reward(self):
        return NotImplementedError

    @property
    def positions_space(self):
        return NotImplementedError

    @abstractmethod
    def reset(self):
        pass

    @abstractmethod
    def render(self):
        pass

    @abstractmethod
    def step(self, action):
        pass


In [7]:
from collections import deque

import cv2
import gym
import numpy as np
from gym import spaces
from tensorflow import keras

# from .game_interface import GameInterface

FRAMES_IN_OBSERVATION = 4
FRAME_SIZE = 84
# if keras.backend.image_data_format() == 'channels_first':
#     # INPUT_SHAPE = (FRAMES_IN_OBSERVATION, FRAME_SIZE, FRAME_SIZE)
#     INPUT_SHAPE = (1, FRAMES_IN_OBSERVATION * FRAME_SIZE, FRAME_SIZE)
# else:
#     INPUT_SHAPE = (FRAME_SIZE * FRAMES_IN_OBSERVATION, FRAME_SIZE, 1)


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = spaces.Box(low=0, high=255, shape=(84, 84, 1))

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not belive how complex the previous solution was."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=0)
        if dtype is not None:
            out = out.astype(dtype)
        return out


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0] * k, shp[1], shp[2]))

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class MainGymWrapper:
    @staticmethod
    def wrap(env):
        env = ProcessFrame84(env)
        env = FrameStack(env, 4)
        return env


class Breakout():
    def __init__(self):
      
      pygame, video_recorder=wrap_env(MainGymWrapper.wrap(gym.make("BreakoutDeterministic-v4", render_mode='rgb_array')))
      self._pygame = pygame
      self.video_recorder=video_recorder

      self.action_space = {key: val for key, val in enumerate(range(pygame.action_space.n))}
      self.state = None  # where is the ninja
      self.total_reward = 0
      self.positions_space = self._pygame.observation_space.shape  # inside reward

    def reset(self):
        self.total_reward = 0
        self.state = self._pygame.reset()
        return self.state

    def render(self):
        self.video_recorder.capture_frame()
        # self._pygame.render()

    def step(self, action):
        next_state, reward, terminal, info = self._pygame.step(action)
        self.state = next_state

        self.total_reward += reward
        return next_state, reward, terminal, info


/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

# 4. RL con Atari

In [8]:
import random

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.optimizers import RMSprop


class NNLearner():
    def __init__(self, game):
        self.state_size = game.positions_space

        self.grid_size = np.prod(game.positions_space)
        self.action_size = len(game.action_space)
        self.memory = list()  # deque(maxlen=2000)
        self.max_memory = 2000
        self.learning_rate = 0.1  # discount rate
        self.ratio_explotacion = 0.8  # 0.95# 0.8  # exploration rate
        self.explotacion_min = 0.05
        self.explotacion_decay = 0.995
        self.model_learning_rate = 0.001
        self.discount_factor = 0.1
        self.model = self._crear_modelo()
        self.game = game
        self.update_iteration = 0

    def _crear_modelo(self):
        model = Sequential()
        model.add(Conv2D(32,
                              8,
                              strides=(4, 4),
                              padding="valid",
                              activation="relu",
                              input_shape=self.state_size,
                              data_format="channels_last"))
        model.add(Conv2D(64,
                              4,
                              strides=(2, 2),
                              padding="valid",
                              activation="relu",
                              input_shape=self.state_size,
                              data_format="channels_last"))
        model.add(Conv2D(64,
                              3,
                              strides=(1, 1),
                              padding="valid",
                              activation="relu",
                              input_shape=self.state_size,
                              data_format="channels_last"))
        model.add(Flatten())
        model.add(Dense(512, activation="relu"))
        model.add(Dense(self.action_size))
        model.compile(loss="mean_squared_error",
                           optimizer=RMSprop(learning_rate=0.00025,
                                             rho=0.95,
                                             epsilon=0.01),
                           metrics=["accuracy"])
        print(model.summary())
        return model

    def update(self, game, old_state, action_taken, reward_action_taken, new_state, new_action, reached_end):
        self.update_iteration += 1
        if reached_end or self.update_iteration >= 999:  # entrenamos este epoch
            self.remember(old_state, action_taken, reward_action_taken,
                          new_state, reached_end)
            self.aprendizaje(min(100, len(self.memory)))
            self.update_iteration = 0

        else:
            # guardamos entrenamiento
            self.remember(old_state, action_taken, reward_action_taken,
                          new_state, reached_end)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append(
            (self.prepare_state(state), self.prepare_action(action), reward, self.prepare_state(next_state), done))
        if len(self.memory) > self.max_memory:
            del self.memory[0]  # se quita el primero

    def get_next_step(self, state, game):
        next_step = np.random.choice(list(game.action_space))
        if np.random.uniform() <= self.ratio_explotacion:
            q = self.model.predict(self.prepare_state(state))
            idx_action = np.argmax(q[0])
            next_step = list(game.action_space)[idx_action]
        return next_step

    def aprendizaje(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        minibatch.append(self.memory[-1])  # final estate
        for state, action, reward, next_state, done in minibatch:
            actual_q_value_options = self.model.predict(state)
            actual_q_value = actual_q_value_options[0][action]

            future_max_q_value = reward
            if not done:
                future_max_q_value = reward + self.discount_factor * np.amax(self.model.predict(next_state)[0])

            actual_q_value_options[0][action] = actual_q_value + self.learning_rate * (
                    future_max_q_value - actual_q_value)
            self.model.fit(state, actual_q_value_options, epochs=1, verbose=0)

        # change exploration rate
        if 1 - self.ratio_explotacion > self.explotacion_min:
            ratio_exploracion = 1 - self.ratio_explotacion
            ratio_exploracion *= self.explotacion_decay
            self.ratio_explotacion = 1 - ratio_exploracion

    def prepare_state(self, state):
        return np.expand_dims(np.asarray(state).astype(np.float64), axis=0)

    def prepare_action(self, action_taken):
        return (list(self.game.action_space).index(action_taken))

    def print_policy(self):
        array_1d = np.zeros(self.grid_size)
        tabs = 0
        for i in range(0, self.grid_size):
            array_1d[i] = 1
            sep = ' '
            if not ((i + 1) % self.game.positions_space[1]):
                # tabs += 1
                sep = '\n'
            print(self.model.predict(array_1d.reshape((1, -1))), end=sep)
            array_1d[i] = 0


In [ ]:
!rm -rf video
learner=NNLearner(Breakout())
max_points = -9999
first_max_reached = 0
total_rw = 0
RENDER=True

for played_games in range(0, 10):  # 100
    game = Breakout()#generamos de nuevo para poder generar un video cada vez
    learner.game = game
    # game._pygame.close()
    # game = Breakout()    
    state = game.reset()
    reward, info, done = None, None, None
    iteration = 0
    while not done and iteration < 10000:  # 10000
        if iteration%50==0:
          print(f"reward:{game.total_reward}",sep="")
        print(".",sep="",end="")
        old_state = np.array(state)
        next_action = learner.get_next_step(state, game)
        state, reward, done, info = game.step(next_action)
        # For sarsa
        next_post_action = learner.get_next_step(state, game)
        learner.update(game, old_state, next_action, reward, state, next_post_action, done)
        if RENDER:
            game.render()
        iteration += 1

    total_rw += game.total_reward
    if game.total_reward > max_points:
        max_points = game.total_reward
        first_max_reached = played_games
    print("\n******** played_games[", played_games, "] Points[", game.total_reward, "]  Steps[", iteration,
          "] MaxPoint[",
          max_points, "]")
    game.video_recorder.close()

/content ./video/fe4a637a-dc3b-11ec-85b2-0242ac1c0002.mp4
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 83, 20, 32)        2080      
                                                                 
 conv2d_7 (Conv2D)           (None, 40, 9, 64)         32832     
                                                                 
 conv2d_8 (Conv2D)           (None, 38, 7, 64)         36928     
                                                                 
 flatten_2 (Flatten)         (None, 17024)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               8716800   
                                                                 
 dense_5 (Dense)             (None, 4)                 2052      
                                                              

/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitoring/video_recorder.py:422: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np.__version__


.................................................reward:0.0
..................................................reward:1.0
..................................................reward:2.0
.............

In [ ]:
game.video_recorder.close()
game._pygame.close()

In [ ]:
show_video()